## Final Project Sravanthi Mummadi


## Initializing Spark Session

In [184]:
# from pyspark.sql import SparkSession

# ama_rev = SparkSession \
#     .builder \
#     .appName("Python Spark SQL basic example") \
#     .config("spark.some.config.option", "some-value") \
#     .getOrCreate()

from pyspark.sql import SparkSession
app_name = "Spark Final Project"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName("Python Spark SQL Final project")\
        .master(master)\
        .getOrCreate()
#sc = spark.sparkContext

## Data Description

Data set we are using for this project is the reviews for the cell phones sold at Amazon website. The dataset was available in two excel files one was for 
reviews: it has 165630 rows and columns like asin (identifier for the products), name(customer name), rating(how much they rated the product), body (the actual reviews provided by them), etc..
items: it had details of the products like brand (cell phone brand), prices, asin, etc..
In our analysis we tried to find answers for few business questions like the brand with good rating? how much product is being sold by each brand?

## Data Loading

In [249]:
ama_revdf = spark.read.options(header="true",\
                              inferSchema="true",\
                              nullValue="NA",\
                              timestampFormat="yyyy-MM-dd'T'HH:mm:ss")\
                             .csv("/Users/sravanthi/Documents/amazon+_reviews.csv")


In [256]:
ama_revdf.count()

165630

In [259]:
ama_revdf.rdd.getNumPartitions()

14

In [251]:
ama_revdf.show()

+----------+--------------------+------+------------------+--------+--------------------+--------------------+------------+
|      asin|                name|rating|              date|verified|               title|                body|helpfulVotes|
+----------+--------------------+------+------------------+--------+--------------------+--------------------+------------+
|B0000SX2UC|               Janet|     3|  October 11, 2005|   false|Def not best, but...|I had the Samsung...|           1|
|B0000SX2UC|          Luke Wyatt|     1|   January 7, 2004|   false|Text Messaging Do...|"Due to a softwar...|     however|
|B0000SX2UC|              Brooke|     5| December 30, 2003|   false|     Love This Phone|This is a great, ...|           5|
|B0000SX2UC|       amy m. teague|     3|    March 18, 2004|   false|Love the Phone, B...|I love the phone ...|           1|
|B0000SX2UC|       tristazbimmer|     4|   August 28, 2005|   false|Great phone servi...|The phone has bee...|           1|
|B0000SX

In [250]:
ama_revdf.printSchema()

root
 |-- asin: string (nullable = true)
 |-- name: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- title: string (nullable = true)
 |-- body: string (nullable = true)
 |-- helpfulVotes: string (nullable = true)



In [139]:
# Imported the Items data
ama_phdf = spark.read.options(header="true",\
                              inferSchema="true",\
                              nullValue="NA",\
                              timestampFormat="yyyy-MM-dd'T'HH:mm:ss",\
                              mode="failfast")\
                             .csv("/Users/sravanthi/Documents/amazon_phones.csv")


In [140]:
# dropped the columns which are repeating
drop_lst = ["title", "rating"]
ama_phdf = ama_phdf.drop(*drop_lst)



In [257]:
ama_phdf.count()

792

In [141]:
#Items dataframe 
ama_phdf.show()

+----------+--------+--------------------+--------------------+--------------------+------------+-------+
|      asin|   brand|                 url|               image|           reviewUrl|totalReviews| prices|
+----------+--------+--------------------+--------------------+--------------------+------------+-------+
|B0000SX2UC|   Nokia|https://www.amazo...|https://m.media-a...|https://www.amazo...|          14|   null|
|B0009N5L7K|Motorola|https://www.amazo...|https://m.media-a...|https://www.amazo...|           7| $49.95|
|B000SKTZ0S|Motorola|https://www.amazo...|https://m.media-a...|https://www.amazo...|          22|   null|
|B00198M12M|   Nokia|https://www.amazo...|https://m.media-a...|https://www.amazo...|           5|   null|
|B001AO4OUC|Motorola|https://www.amazo...|https://m.media-a...|https://www.amazo...|          21|   null|
|B001DCJAJG|Motorola|https://www.amazo...|https://m.media-a...|https://www.amazo...|          12|   null|
|B001DZY4KI|    Sony|https://www.amazo...|http

In [260]:
ama_phdf.rdd.getNumPartitions()

1

In [142]:
#Schema of the Itmes
ama_phdf.printSchema()

root
 |-- asin: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- url: string (nullable = true)
 |-- image: string (nullable = true)
 |-- reviewUrl: string (nullable = true)
 |-- totalReviews: string (nullable = true)
 |-- prices: string (nullable = true)



## Data Cleaning

In [143]:
# cleaned the Prices column by removing special characters.
from pyspark.sql import functions as F
from pyspark.sql.functions import when  

cols = F.split(ama_phdf['prices'], ',')
ama_phdf = ama_phdf.withColumn('prices', cols.getItem(0))

In [144]:
from  pyspark.sql.functions import regexp_replace,col
ama_phdf = ama_phdf.withColumn("prices",F.regexp_replace(F.col("prices"), "[\$]", ""))



In [145]:
# changed the prices column data type
ama_phdf = ama_phdf.withColumn("prices", ama_phdf.prices.cast("int"))

In [146]:
ama_phdf.show()

+----------+--------+--------------------+--------------------+--------------------+------------+------+
|      asin|   brand|                 url|               image|           reviewUrl|totalReviews|prices|
+----------+--------+--------------------+--------------------+--------------------+------------+------+
|B0000SX2UC|   Nokia|https://www.amazo...|https://m.media-a...|https://www.amazo...|          14|  null|
|B0009N5L7K|Motorola|https://www.amazo...|https://m.media-a...|https://www.amazo...|           7|    49|
|B000SKTZ0S|Motorola|https://www.amazo...|https://m.media-a...|https://www.amazo...|          22|  null|
|B00198M12M|   Nokia|https://www.amazo...|https://m.media-a...|https://www.amazo...|           5|  null|
|B001AO4OUC|Motorola|https://www.amazo...|https://m.media-a...|https://www.amazo...|          21|  null|
|B001DCJAJG|Motorola|https://www.amazo...|https://m.media-a...|https://www.amazo...|          12|  null|
|B001DZY4KI|    Sony|https://www.amazo...|https://m.med

In [189]:
# created a year column and seperated the year from date for further analysis
from pyspark.sql.functions import lit
ama_revdf = ama_revdf.withColumn("Year", lit(0))


In [190]:
cols = F.split(ama_revdf['date'], ',')
ama_revdf = ama_revdf.withColumn('Year', cols.getItem(1))


In [231]:
ama_revdf = ama_revdf.withColumn("Year", ama_revdf.Year.cast("int"))

## Join DataFrames

In [234]:
# joined the 2 tables with column column
col_list=["asin"]
amz_join = ama_revdf.join(ama_phdf, col_list, "inner")

In [235]:
# after creation schema
amz_join.printSchema()

amz_join.count()

root
 |-- asin: string (nullable = true)
 |-- name: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- verified: boolean (nullable = true)
 |-- title: string (nullable = true)
 |-- body: string (nullable = true)
 |-- helpfulVotes: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- brand: string (nullable = true)
 |-- url: string (nullable = true)
 |-- image: string (nullable = true)
 |-- reviewUrl: string (nullable = true)
 |-- totalReviews: string (nullable = true)
 |-- prices: integer (nullable = true)



165630

## Spark SQL

#### what is the proportion of mobiles sold by different brands in the  Amazon Market

In [196]:
# Register the DataFrame as a SQL temporary view
amz_join.createOrReplaceTempView("amz_join")

# Q1) Find the number of mobiles sold by each brand in amazon?

sql_df = spark.sql('select brand,count(name) as count from amz_join group by brand')
sql_df.show()
              
type(sql_df)

+--------+-----+
|   brand|count|
+--------+-----+
|   Nokia|11520|
|    Sony| 6768|
|Motorola|17684|
|  Xiaomi| 5896|
| Samsung|83544|
|  Google| 8058|
| OnePlus| 1126|
|    ASUS| 1008|
|   Apple|24030|
|  HUAWEI| 5992|
+--------+-----+



pyspark.sql.dataframe.DataFrame

In [80]:
import plotly as py
import plotly.graph_objs as go
import pandas as pd
import requests

In [197]:
import plotly
import plotly.graph_objs as go
import plotly.offline as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

data = [go.Pie(labels=sql_df.toPandas()['brand'],values= sql_df.toPandas()['count'], hole=.3)]
# Edit the layout
layout = dict(title = "Pie chart of product sold by each brand",
              xaxis = dict(title = 'brand'),
              yaxis = dict(title = 'count'),
              )
fig = dict(data=data, layout=layout)
py.iplot(fig)

### What is the average rating given by customers for each brand?

In [253]:

# Q2)Find the average rating given by customers for each brand?

sql_df2 = spark.sql('select brand, Avg(rating) as avg_rating from amz_join \
group by brand order by avg_rating desc ')
sql_df2.show()

+--------+------------------+
|   brand|        avg_rating|
+--------+------------------+
|  Xiaomi| 4.350746268656716|
|  HUAWEI| 4.312645979312646|
| OnePlus|3.8898756660746003|
|Motorola| 3.774259217371635|
| Samsung|3.7476120939362745|
|    Sony|3.7163120567375887|
|    ASUS|3.7123015873015874|
|   Apple|3.6410320432792345|
|  Google|3.6366344005956814|
|   Nokia|3.5947916666666666|
+--------+------------------+



In [262]:
# Bar plot for average rating for different brand.

data = go.Data([go.Bar(x=sql_df2.toPandas()['avg_rating'],
                y=sql_df2.toPandas()['brand'],
                orientation='h',
                name="AVERAGE RATING",
                marker=dict(color='#9467bd'))])
# Edit the layout
layout = dict(title = "AVERAGE RATING PER BRAND",
              xaxis = dict(title = 'Avgerage Rating'),
              yaxis = dict(title = 'Brand')
              )
fig = dict(data=data, layout=layout)
py.iplot(fig)


### What is the Maximum and minimum price of mobiles sold by each brand.

In [204]:
# Q3) Find the Maximum and minimum price of mobiles sold by each brand.

# Register the DataFrame as a SQL temporary view
amz_join.createOrReplaceTempView("ama_phdf")

# spark.sql("select unique(gender) from surveydf")
sql_phdf = spark.sql('Select brand,max(prices) as max_price ,min(prices) as min_price from ama_phdf where prices >= 50 group  by brand')
sql_phdf.show()

# branddf = spark.sql("Select brand,avg(rating),max(prices),min(prices) from amz_join group  by brand")
# branddf.show()

+--------+---------+---------+
|   brand|max_price|min_price|
+--------+---------+---------+
|   Nokia|      570|       56|
|    Sony|      499|       78|
|Motorola|      499|       59|
|  Xiaomi|      413|       57|
| Samsung|      910|       54|
|  Google|      679|      107|
| OnePlus|      829|      293|
|    ASUS|      359|       79|
|   Apple|      999|       79|
|  HUAWEI|      809|       59|
+--------+---------+---------+



In [207]:

# Side by side bar plot for minimum and maximum prices for different brands.


init_notebook_mode(connected=True)
trace_a = go.Bar(x=sql_phdf.toPandas()['brand'],
                y=sql_phdf.toPandas()['min_price'],
                name="Minimum_Prices",
                marker=dict(color='#A2D5F2'))

trace_b = go.Bar(x=sql_phdf.toPandas()['brand'],
                y=sql_phdf.toPandas()['max_price'],
                name="Maximum_prices",
                marker=dict(color='#FFCDD2'),
              )
    

data3 = go.Data([trace_a,trace_b])

# Edit the layout
layout = dict(title = "Minimum and Maximum prices for different brands",
              xaxis = dict(title = 'brand'),
              yaxis = dict(title = 'Prices'),
              )

fig = dict(data=data3, layout=layout)
py.iplot(fig)



### What are the Count of satisfied consumers who gave top rating(ratings greater than 4)for each brand

In [255]:
#  Q4) Count of satisfied consumers who gave top rating(ratings greater than 4)for each brand

tot_rev = spark.sql("Select brand,count(rating) as above4 from amz_join where \
rating >= 4 group  by brand order by above4 desc")
tot_rev.show()


+--------+------+
|   brand|above4|
+--------+------+
| Samsung| 55914|
|   Apple| 15466|
|Motorola| 11986|
|   Nokia|  7204|
|  Google|  5112|
|  HUAWEI|  4954|
|  Xiaomi|  4922|
|    Sony|  4428|
| OnePlus|   800|
|    ASUS|   658|
+--------+------+



In [254]:
prop_4 = spark.sql("Select t.brand, (r.reviews/t.total)*100 as proportion_above4 from \
(Select brand, count(rating) as total from amz_join group by brand) as t join (Select brand,count(rating) as reviews \
from amz_join where rating>= 4 group by brand) as r on t.brand=r.brand order by proportion_above4 desc")
prop_4.show()

+--------+-----------------+
|   brand|proportion_above4|
+--------+-----------------+
|  Xiaomi|83.48032564450475|
|  HUAWEI|82.64931598264931|
| OnePlus|71.04795737122558|
|Motorola|67.77878308075096|
| Samsung| 66.9260048356594|
|    Sony|65.42553191489363|
|    ASUS|65.27777777777779|
|   Apple|  64.361215147732|
|  Google|63.44005956813105|
|   Nokia|62.53472222222223|
+--------+-----------------+



In [230]:
# bar plot for count of reviews for each brand
data = go.Data([go.Bar(x=prop_4.toPandas()['brand'],
                y=prop_4.toPandas()['proportion_above4'],
                name="Atleast 4 RATING",
                marker=dict(color='#A2D5F2'))])
# Edit the layout
layout = dict(title = "Count of reviews with Min 4 rating",
              xaxis = dict(title = 'brand'),
              yaxis = dict(title = 'above 4 rating')
              )
fig = dict(data=data, layout=layout)
py.iplot(fig)


### What are number of phones sold by Brands for each year? 

In [264]:
#Q6) Count of phones sold by Brands for each year? 
sql_area = spark.sql("Select year,count(asin),brand from amz_join group by brand, year order by year")
sql_area.show()

+-----+-----------+--------+
| year|count(asin)|   brand|
+-----+-----------+--------+
| 2003|          4|   Nokia|
| 2004|         18|   Nokia|
| 2005|          6|   Nokia|
| 2005|          2|Motorola|
| 2006|          4|Motorola|
| 2007|         10|Motorola|
| 2008|          8|Motorola|
| 2008|          6|   Nokia|
| 2009|         26|Motorola|
| 2009|          4|   Nokia|
| 2009|         16| Samsung|
| 2010|        102| Samsung|
| 2010|         18|Motorola|
| 2011|        218| Samsung|
| 2011|         50|   Nokia|
| 2011|          2|    Sony|
| 2011|         40|Motorola|
| 2012|        234|Motorola|
| 2012|         26|  HUAWEI|
| 2012|        766| Samsung|
+-----+-----------+--------+
only showing top 20 rows



In [156]:
# sql_area_pandas = sql_area.toPandas()
import plotly.express as px
#area = px.sql_area_pandas.gapminder()
fig = px.area(sql_area.toPandas(), x="year", y="count(asin)", color="brand")
fig.show()
